In [23]:
import json
import pandas
import numpy
import os
import sys
from os import path

In [24]:
import json
import os
import pandas
import pyarrow

from dotenv import load_dotenv
from sqlalchemy import create_engine

# Steps to install
# 1. pip install sqlalchemy-bigquery google-cloud-bigquery-storage pyarrow
# 2. Copy the credentials file to wherever you set BIGQUERY_CREDENTIALS_PATH to

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')

In [25]:
sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
import create_queries
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
COMPANY_NAME = 'HPCC'
COMPANY_IDENTIFIER = 'HPCC'
TRANSFER_PACKAGES_START_DATE = '2020-01-01'
SALES_TRANSACTIONS_START_DATE = '2020-01-01'
INVENTORY_DATES = [
#     '09/30/2020',
#     '10/31/2020',
#     '11/30/2020',
#     '12/31/2020',
#     '01/31/2021',
#     '02/28/2021',
#     '03/31/2021',
#     '04/30/2021',
#     '05/31/2021',
#     '06/30/2021',
#     '07/31/2021',
#     '08/31/2021',
    '09/30/2021',
]
INVENTORY_DATE = '10/11/2021'

In [27]:
company_incoming_transfer_packages_query = create_queries.create_company_incoming_transfer_packages_query(COMPANY_IDENTIFIER, TRANSFER_PACKAGES_START_DATE)
company_outgoing_transfer_packages_query = create_queries.create_company_outgoing_transfer_packages_query(COMPANY_IDENTIFIER, TRANSFER_PACKAGES_START_DATE)
company_sales_transactions_query = create_queries.create_company_sales_transactions_query(COMPANY_IDENTIFIER, SALES_TRANSACTIONS_START_DATE)
company_inventory_packages_query = create_queries.create_company_inventory_packages_query(COMPANY_IDENTIFIER)

engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))
company_incoming_transfer_packages_dataframe = pandas.read_sql_query(company_incoming_transfer_packages_query, engine)
company_outgoing_transfer_packages_dataframe = pandas.read_sql_query(company_outgoing_transfer_packages_query, engine)
company_sales_transactions_dataframe = pandas.read_sql_query(company_sales_transactions_query, engine)
company_inventory_packages_dataframe = pandas.read_sql_query(company_inventory_packages_query, engine)

In [28]:
company_incoming_transfer_packages_dataframe

,package_row_id,delivery_type,license_number,manifest_number,created_date,received_datetime,shipper_facility_license_number,shipper_facility_name,recipient_facility_license_number,recipient_facility_name,...,product_category_name,product_name,package_lab_results_status,shipper_wholesale_price,shipped_quantity,shipped_unit_of_measure,received_quantity,received_unit_of_measure,item_unit_weight,item_unit_weight_unit_of_measure_name
0,6b0924d3-197c-47ae-aa0f-b0cd7c193511,INCOMING_FROM_VENDOR,C10-0000064-LIC,0002560036,2021-10-11,NaT,C11-0000127-LIC,"INDUSTRIAL COURT L11, LLC",C10-0000064-LIC,"HUENEME PATIENT CONSUMER COLLECTIVE, LLC.",...,Pre-Roll Infused,Grand CRU 40s-0.5g Infused PreRoll-GMO Cookies,passed,237.5,50.0,Each,NaN,None,0.5,Grams
1,6379ca03-3b12-49fc-bc2e-5ebc7fd3293d,INCOMING_FROM_VENDOR,C10-0000064-LIC,0002553220,2021-10-09,NaT,C12-0000381-LIC,"THE TAHOE GROUP, LLC",C10-0000064-LIC,"HUENEME PATIENT CONSUMER COLLECTIVE, LLC.",...,Pre-Roll Infused,Gorilla Cake 4-Pack Infused,passed,21.0,25.0,Each,NaN,None,4.0,Grams
2,9d650908-a1c0-4ddd-9617-2956f844f53a,INCOMING_FROM_VENDOR,C10-0000064-LIC,0002553220,2021-10-09,NaT,C12-0000381-LIC,"THE TAHOE GROUP, LLC",C10-0000064-LIC,"HUENEME PATIENT CONSUMER COLLECTIVE, LLC.",...,Pre-Roll Infused,2X Wedding Cake Drippy J 1.5G,passed,7.5,40.0,Each,NaN,None,1.5,Grams
3,2b43ae7b-4970-4968-bdff-4b14c34ad194,INCOMING_FROM_VENDOR,C10-0000064-LIC,0002553220,2021-10-09,NaT,C12-0000381-LIC,"THE TAHOE GROUP, LLC",C10-0000064-LIC,"HUENEME PATIENT CONSUMER COLLECTIVE, LLC.",...,Vape Cartridge (weight - each),Lemon OG 1G Cart,passed,13.0,40.0,Each,NaN,None,1.0,Grams
4,f7c39f62-c064-4f37-8299-936cbd908a92,INCOMING_FROM_VENDOR,C10-0000064-LIC,0002553220,2021-10-09,NaT,C12-0000381-LIC,"THE TAHOE GROUP, LLC",C10-0000064-LIC,"HUENEME PATIENT CONSUMER COLLECTIVE, LLC.",...,Vape Cartridge (weight - each),Banana OG 1G Cart,passed,13.0,50.0,Each,NaN,None,1.0,Grams
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36017,c9996636-50f4-41aa-aeb5-343e465af6b0,INCOMING_FROM_VENDOR,C10-0000064-LIC,0000226935,2020-01-02,2020-01-03 22:00:13+00:00,C11-0000578-LIC,"9701 ENTERPRISES, INC.",C10-0000064-LIC,"HUENEME PATIENT CONSUMER COLLECTIVE, LLC.",...,Flower,Kaviar Moonrocks - Indica - 1g / Case of 28,failed,280.0,28.0,Grams,28.0,Grams,NaN,None
36018,69304746-8367-4ec9-8be1-b6b5f85cd3b4,INCOMING_FROM_VENDOR,C10-0000064-LIC,0000226935,2020-01-02,2020-01-03 22:00:13+00:00,C11-0000578-LIC,"9701 ENTERPRISES, INC.",C10-0000064-LIC,"HUENEME PATIENT CONSUMER COLLECTIVE, LLC.",...,Flower,Kaviar Moonrocks - Indica - 1g / Case of 28,failed,280.0,28.0,Grams,28.0,Grams,NaN,None
36019,dd3837aa-871e-4fa0-98f7-1841c812781a,INCOMING_FROM_VENDOR,C10-0000064-LIC,0000226935,2020-01-02,2020-01-03 22:00:13+00:00,C11-0000578-LIC,"9701 ENTERPRISES, INC.",C10-0000064-LIC,"HUENEME PATIENT CONSUMER COLLECTIVE, LLC.",...,Flower,Kaviar Moonrock - Hybrid - 1g / Case of 28,failed,280.0,28.0,Grams,28.0,Grams,NaN,None
36020,30846e39-0191-4b39-a2a0-90b9afc75784,INCOMING_FROM_VENDOR,C10-0000064-LIC,0000226935,2020-01-02,2020-01-03 22:00:13+00:00,C11-0000578-LIC,"9701 ENTERPRISES, INC.",C10-0000064-LIC,"HUENEME PATIENT CONSUMER COLLECTIVE, LLC.",...,Flower,Kaviar Moonrocks - Indica - 1g / Case of 28,failed,280.0,28.0,Grams,28.0,Grams,NaN,None


In [29]:
company_outgoing_transfer_packages_dataframe

,package_row_id,delivery_type,license_number,manifest_number,created_date,received_datetime,shipper_facility_license_number,shipper_facility_name,recipient_facility_license_number,recipient_facility_name,...,product_category_name,product_name,package_lab_results_status,shipper_wholesale_price,shipped_quantity,shipped_unit_of_measure,received_quantity,received_unit_of_measure,item_unit_weight,item_unit_weight_unit_of_measure_name
0,363e44f9-4ea5-4438-b580-2d70b530bc37,OUTGOING_TO_PAYOR,C10-0000064-LIC,0002485676,2021-09-23,NaT,C10-0000064-LIC,"HUENEME PATIENT CONSUMER COLLECTIVE, LLC.",C11-0000139-LIC,APEX SOLUTIONS,...,Vape Cartridge (weight - each),AiroPro Strain Series Pineapple Diesel 1g,passed,None,1.0,Each,NaN,None,1.0,Grams
1,e8801bf3-2fda-47b1-8a08-36f243d381eb,OUTGOING_TO_PAYOR,C10-0000064-LIC,0002485676,2021-09-23,NaT,C10-0000064-LIC,"HUENEME PATIENT CONSUMER COLLECTIVE, LLC.",C11-0000139-LIC,APEX SOLUTIONS,...,Vape Cartridge (weight - each),AiroPro Strain Series Strawberry Cheesecake 1g,passed,None,1.0,Each,NaN,None,1.0,Grams
2,664d8684-c7ff-46a2-a9cd-bc2bc287b68c,OUTGOING_TO_PAYOR,C10-0000064-LIC,0002485676,2021-09-23,NaT,C10-0000064-LIC,"HUENEME PATIENT CONSUMER COLLECTIVE, LLC.",C11-0000139-LIC,APEX SOLUTIONS,...,Vape Cartridge (weight - each),AiroPro Strain Series Clementine 1g,passed,None,1.0,Each,NaN,None,1.0,Grams
3,bf1b8d59-b069-474e-983d-c51dfb56f5fd,OUTGOING_TO_PAYOR,C10-0000064-LIC,0002485676,2021-09-23,NaT,C10-0000064-LIC,"HUENEME PATIENT CONSUMER COLLECTIVE, LLC.",C11-0000139-LIC,APEX SOLUTIONS,...,Vape Cartridge (weight - each),AiroPro Strain Series Pineapple Diesel 0.5g,passed,None,1.0,Each,NaN,None,0.5,Grams
4,d2d384d5-b96b-4093-8715-90473f6b6f98,OUTGOING_TO_PAYOR,C10-0000064-LIC,0002485676,2021-09-23,NaT,C10-0000064-LIC,"HUENEME PATIENT CONSUMER COLLECTIVE, LLC.",C11-0000139-LIC,APEX SOLUTIONS,...,Vape Cartridge (weight - each),AiroPro Artisan Series Black Mamba 0.5g,passed,None,1.0,Each,NaN,None,0.5,Grams
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,c1a2eec6-a309-4829-95e8-4cd83c5afd94,OUTGOING_TO_PAYOR,C10-0000064-LIC,0000304408,2020-02-06,2020-02-10 23:49:53+00:00,C10-0000064-LIC,"HUENEME PATIENT CONSUMER COLLECTIVE, LLC.",C11-0000620-LIC,IRONWORKS COLLECTIVE INC.,...,Vape Cartridge (volume - each),STIIIZY - Blue Burst Pod - 1.0g,passed,None,15.0,Each,15.0,Each,NaN,None
251,d93a1c1e-3d98-4928-867f-41616eccbb72,OUTGOING_TO_PAYOR,C10-0000064-LIC,0000303610,2020-02-05,2020-02-08 01:36:29+00:00,C10-0000064-LIC,"HUENEME PATIENT CONSUMER COLLECTIVE, LLC.",C11-0000127-LIC,"INDUSTRIAL COURT L11, LLC",...,Flower,Littles 3.5g Flower - Sativa,failed,None,441.0,Grams,441.0,Grams,NaN,None
252,ae692851-76db-4948-8297-2821b2526347,OUTGOING_TO_PAYOR,C10-0000064-LIC,0000245385,2020-01-10,2020-01-10 20:23:01+00:00,C10-0000064-LIC,"HUENEME PATIENT CONSUMER COLLECTIVE, LLC.",C11-0000258-LIC,YERBA BUENA LOGISTICS SERVICES,...,Other Concentrate (weight - each),Beg Inv_DipnDabs_ISO_Lemon OG 1g,failed,None,19.0,Each,19.0,Each,1.0,Grams
253,60f66f5f-4c53-4844-9f92-47389385badc,OUTGOING_TO_PAYOR,C10-0000064-LIC,0000241041,2020-01-09,2020-01-09 02:50:07+00:00,C10-0000064-LIC,"HUENEME PATIENT CONSUMER COLLECTIVE, LLC.",C11-0000760-LIC,"GREEN SPECTRUM TRADING, INC.",...,Leaf,wedding crasher trim,failed,None,7500.0,Grams,7500.0,Grams,NaN,None


In [30]:
company_sales_transactions_dataframe

,id,license_number,receipt_number,rt_type,sales_customer_type,sales_datetime,total_packages,rt_total_price,tx_type,tx_package_id,tx_package_label,tx_product_name,tx_product_category_name,tx_unit_of_measure,tx_quantity_sold,tx_total_price
0,93f00b8d-bba5-4872-8181-c20f19779e00,C10-0000064-LIC,0135603109,active,Consumer,2021-10-11 21:02:48+00:00,1,31.26,active,17956638,1A4060300003D59000020075,Rich & Ruthless - Purple Hulk (3.5 GM Flower (...,Flower (packaged eighth - each),Each,2.0,31.26
1,d4bd48f1-501b-46e9-bb76-4d6664dbc4e5,C10-0000064-LIC,0135602813,active,Consumer,2021-10-11 21:01:34+00:00,3,33.43,active,18498078,1A4060300006D67000049188,NFuzed - Sour Gummy BB_ - 5.6g,Edible (weight - each),Each,1.0,0.01
2,d4bd48f1-501b-46e9-bb76-4d6664dbc4e5,C10-0000064-LIC,0135602813,active,Consumer,2021-10-11 21:01:34+00:00,3,33.43,active,18498080,1A4060300006D67000049190,NFuzed - THC Sour Gummies Variety Pack_ - 56g,Edible (weight - each),Each,1.0,9.38
3,d4bd48f1-501b-46e9-bb76-4d6664dbc4e5,C10-0000064-LIC,0135602813,active,Consumer,2021-10-11 21:01:34+00:00,3,33.43,active,18578963,1A406030000A2BE000002861,APE - Ice Cream Cake 1.1G,Vape Cartridge (weight - each),Each,1.0,24.04
4,8cba10e6-503f-4b04-89ce-7c711e0eb874,C10-0000064-LIC,0135602020,active,Consumer,2021-10-11 20:57:04+00:00,2,152.87,active,18426536,1A406030000F99E000218278,KIVA - Kiva Bar White Chocolate Raspberries an...,Edible (weight - each),Each,1.0,22.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
784192,59647e41-3637-4b4c-b9cd-3fe4c6685c3b,C10-0000064-LIC,0005126710,inactive,Consumer,2020-01-01 10:10:40+00:00,2,63.31,inactive,1689409,1A40603000078B5000006999,Big Pete's:Chocolate Chip Indica 10pk Cookie 1...,Edible (weight - each),Each,1.0,17.59
784193,fa2035f7-b1a2-4ad9-bbcc-1eb7d579864a,C10-0000064-LIC,0005126711,inactive,Consumer,2020-01-01 10:08:51+00:00,1,29.79,inactive,1576743,1A4060300005276000002017,StrongAg CoastalSun Banjo Packed 1/8,Flower,Grams,3.5,29.79
784194,ac8553c9-94dd-49d4-a438-7fac7fcf78b7,C10-0000064-LIC,0005126700,inactive,Consumer,2020-01-01 10:08:04+00:00,1,18.39,inactive,1725816,1A4060300004F59000003337,Bulk Eighth Brother Blue Dream,Flower,Grams,3.5,18.39
784195,11ab0c03-1423-4f32-959f-e7147f557337,C10-0000064-LIC,0005126709,inactive,Consumer,2020-01-01 10:06:06+00:00,1,18.39,inactive,1725816,1A4060300004F59000003337,Bulk Eighth Brother Blue Dream,Flower,Grams,3.5,18.39


In [31]:
company_inventory_packages_dataframe

,license_number,package_id,package_label,type,packaged_date,package_type,product_name,product_category_name,quantity,unit_of_measure,is_testing_sample,is_trade_sample,is_on_hold,archived_date,finished_date
0,C10-0000064-LIC,18566986,1A4060300003D59000020883,active,2021-10-08,Product,Cali Flwr Farms - Cali Gas OG (1 GM Flower (Ea...,Flower (packaged gram - each),2.0,Each,False,False,False,None,None
1,C10-0000064-LIC,18574336,1A40603000045FF000001847,active,2021-10-08,Product,Strong Ag Coastal Sun Sour Strawberry 1/8 Display,Shake (Packaged Eighth - each),1.0,Each,False,False,False,None,None
2,C10-0000064-LIC,18578964,1A406030000A2BE000002862,active,2021-10-08,Product,APE - Pink Runtz - 1.1g,Vape Cartridge (weight - each),45.0,Each,False,False,False,None,None
3,C10-0000064-LIC,18578962,1A406030000A2BE000002860,active,2021-10-08,Product,APE - Black Mamba 1.1G,Vape Cartridge (weight - each),44.0,Each,False,False,False,None,None
4,C10-0000064-LIC,18566992,1A4060300003D59000020890,active,2021-10-08,Product,Cali Flwr Farms - Cali Ghost (3.5 GM Flower (E...,Flower (packaged eighth - each),150.0,Each,False,False,False,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2572,C10-0000064-LIC,6001675,1A4060300002CED000009490,active,2020-07-23,Product,Face Serum 15ml,Topical (volume - each),1.0,Each,False,False,False,None,None
2573,C10-0000064-LIC,5529480,1A4060300007406000033171,active,2020-07-02,Product,Cartridge - Alien Labs - Disposable - 0.5 Gram...,Vape Cartridge (weight - each),1.0,Each,False,False,False,None,None
2574,C10-0000064-LIC,4701576,1A406030000A4D2000000364,active,2020-05-28,Product,Napalm 1g B52,Vape Cartridge (weight - each),1.0,Each,False,False,False,None,None
2575,C10-0000064-LIC,3396246,1A406030000A2C9000000071,active,2020-03-25,Product,Branded 0.5g Enriched Live Resin XJ-13 Disposa...,Vape Cartridge (weight - each),1.0,Each,False,False,False,None,None


In [70]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:
%autoreload 2

sys.path.append(path.realpath(path.join(os.getcwd(), "../../scripts/analysis")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))

from util import active_inventory_util as util

In [63]:
d = util.Download()
d.download_dataframes(
    incoming_transfer_packages_dataframe=company_incoming_transfer_packages_dataframe,
    outgoing_transfer_packages_dataframe=company_outgoing_transfer_packages_dataframe,
    sales_transactions_dataframe=company_sales_transactions_dataframe,
)

In [80]:
q = util.Query()
q.inventory_dates = INVENTORY_DATES
q.company_name = COMPANY_NAME

id_to_history = util.get_histories(d)
util.print_counts(id_to_history)
util.create_inventory_xlsx(id_to_history, q)

Only outgoing: 199
Only incoming: 8278
In and out: 56
In and sold at least once 27575
In and sold many times 26288
Total pkgs: 37440
WARN: package #5172609 does not have a shipped quantity
Wrote result to out/HPCC_inventory_by_month.xls
Excluded 1735 / 37440 packages from consideration (4.63%)
  MANY_INCOMING: 158 times
  MISSING_INCOMING: 1577 times


In [82]:
computed_inventory_package_records = util.create_inventory_dataframe_by_date(id_to_history, INVENTORY_DATE)
computed_inventory_packages_dataframe = pandas.DataFrame(
    computed_inventory_package_records,
    columns=[
        'package_id',
        'Arrived Date',
        'Product Category',
        'Product Name',
        'Current Quantity',
        'Sold Date',
    ]
)
computed_inventory_packages_dataframe

WARN: package #5172609 does not have a shipped quantity


,package_id,Arrived Date,Product Category,Product Name,Current Quantity,Sold Date
0,18596054,10/09/2021,Vape Cartridge (weight - each),Jack Herer 1G Cart,50,
1,18596053,10/09/2021,Vape Cartridge (weight - each),Lemon OG 1G Cart,40,
2,18596051,10/09/2021,Vape Cartridge (weight - each),Pineapple Express 1G Cart,50,
3,18596058,10/09/2021,Vape Cartridge (weight - each),Wedding Cake Hybrid Vape Cart,40,
4,18596052,10/09/2021,Vape Cartridge (weight - each),Sensi Star 1G Cart,40,
...,...,...,...,...,...,...
25243,1804727,01/03/2020,Vape Cartridge (volume - each),11900 Raw Garden Cartridge 0.5g Indica Fire Wa...,5,
25244,1804719,01/03/2020,Vape Cartridge (volume - each),11962 Raw Garden Cartridge 0.5g Hybrid After P...,7,
25245,1804722,01/03/2020,Vape Cartridge (volume - each),11897 Raw Garden Cartridge 0.5g Indica Autumn ...,8,
25246,1347390,01/02/2020,Flower,Kaviar Moonrocks - Indica - 1g / Case of 28,2,


In [86]:
date_to_inventory_dataframe = {}
for date, inventory_records in date_to_inventory_records.items():
    date_to_inventory_dataframe[date] = pandas.DataFrame(
        inventory_records,
        columns=[
            'package_id',
            'Arrived Date',
            'Product Category',
            'Product Name',
            'Current Quantity',
            'Sold Date',
        ]
    )
    
date_to_inventory_dataframe[list(date_to_inventory_dataframe.keys())[0]]

,package_id,Arrived Date,Product Category,Product Name,Current Quantity,Sold Date
0,8284414,09/30/2020,Flower (packaged eighth - each),3Cs Strawbinati 3.5 jar,0,10/09/2020
1,8269529,09/29/2020,Extract (weight - each),Pink Banana Cake Badder (Gram Jar),20,
2,8275266,09/29/2020,Vape Cartridge (volume - each),13871 Raw Garden Cartridge 1.0g Indica Mendo P...,40,
3,8275261,09/29/2020,Vape Cartridge (volume - each),11294 Raw Garden Cartridge 1.0g Indica Skydogg...,40,
4,8275262,09/29/2020,Vape Cartridge (volume - each),13684 Raw Garden Cartridge 1.0g 1:1 CBD/THC Ar...,20,
...,...,...,...,...,...,...
1459,2975297,02/12/2020,Extract (volume - each),Acapulco Gold Vape Cart,1,
1460,2975671,02/12/2020,Extract (weight - each),831 Extracts Live Resin Sugar Gelato,1,
1461,2609323,02/12/2020,Pre-Roll Infused,AQ Infused PreRoll,4,
1462,2578564,02/12/2020,Flower,THC D- Domo G Indica 3.5 g Jar,7,


In [87]:
raw_incoming_transfer_packages_dataframe = pandas.read_excel('data/20210930/royal_apothecary_incoming_transfer_packages_20200101_20210930.xlsx', header=0)
len(raw_incoming_transfer_packages_dataframe.index), raw_incoming_transfer_packages_dataframe.columns

/Users/warren/venvs/bespoke/lib/python3.7/site-packages/openpyxl/styles/stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


(5484,
 Index(['transfer_row_id', 'delivery_row_id', 'package_row_id', 'delivery_type',
        'license_number', 'manifest_number', 'created_date',
        'transfer_lab_results_status', 'received_datetime',
        'shipper_facility_license_number', 'shipper_facility_name',
        'recipient_facility_license_number', 'recipient_facility_name',
        'shipment_type_name', 'shipment_transaction_type', 'package_id',
        'package_label', 'type', 'product_category_name', 'product_name',
        'shipper_wholesale_price', 'shipped_quantity',
        'package_lab_results_status', 'shipment_package_state',
        'is_testing_sample', 'is_trade_sample'],
       dtype='object'))

In [88]:
incoming_transfer_packages_dataframe = raw_incoming_transfer_packages_dataframe[[
#     'date_type',
#     'transfer_row_id',
#     'delivery_row_id',
#     'package_row_id',
    'delivery_type',
    'manifest_number',
    'created_date',
    'received_datetime',
    'shipper_facility_license_number',
    'shipper_facility_name',
    'recipient_facility_license_number',
    'recipient_facility_name',
    'shipment_type_name',
    'shipment_transaction_type',
    'package_id',
    'package_label',
    'type',
    'product_category_name',
    'product_name',
    'shipper_wholesale_price',
    'shipped_quantity',
    'package_lab_results_status',
    'shipment_package_state',
    'is_testing_sample',
    'is_trade_sample'
]]

In [93]:
for date, inventory_dataframe in date_to_inventory_dataframe.items():
    print(date)
    print(f'# of packages in inventory: {len(inventory_dataframe.index)}')

    inventory_with_incoming_transfer_packages_dataframe = inventory_dataframe.astype({'package_id': 'int64'}).merge(incoming_transfer_packages_dataframe, on='package_id', how='inner', suffixes=('_l', '_r'))
#     print(f'# of packages in inventory with incoming package: {len(inventory_with_incoming_transfer_packages_dataframe.index)}')
    
    inventory_with_cost_records = inventory_with_incoming_transfer_packages_dataframe.to_dict('record')

    total_valuation_cost = 0
    import math
    for inventory_with_cost_record in inventory_with_cost_records:
        incoming_shipped_price = inventory_with_cost_record['shipper_wholesale_price']
        if math.isnan(incoming_shipped_price):
            incoming_shipped_price = 0
        incoming_shipped_quantity = inventory_with_cost_record['shipped_quantity']
        current_quantity = inventory_with_cost_record['Current Quantity']
        total_valuation_cost += float(current_quantity) * (incoming_shipped_price / incoming_shipped_quantity)

    print(f'Inventory valuation (based on COST): ${total_valuation_cost}')
    print('')

09/30/2020
# of packages in inventory: 1464
Inventory valuation (based on COST): $164802.64571975905

10/31/2020
# of packages in inventory: 1713
Inventory valuation (based on COST): $214048.16463415074

11/30/2020
# of packages in inventory: 1671
Inventory valuation (based on COST): $160038.30420620358

12/31/2020
# of packages in inventory: 1813
Inventory valuation (based on COST): $148865.83005506246

01/31/2021
# of packages in inventory: 2070
Inventory valuation (based on COST): $165997.68341993817

02/28/2021
# of packages in inventory: 2232
Inventory valuation (based on COST): $152399.19214778862

03/31/2021
# of packages in inventory: 2389
Inventory valuation (based on COST): $165568.29958287667

04/30/2021
# of packages in inventory: 2516
Inventory valuation (based on COST): $198215.88367231708

05/31/2021
# of packages in inventory: 2644
Inventory valuation (based on COST): $205491.10586772586

06/30/2021
# of packages in inventory: 2824
Inventory valuation (based on COST): $